# Setup

In [1]:
import sys
parent_dir='C:\\Users\\garav\\AGOP\\DLR'
#parent_dir = os.path.abspath(os.path.join(os.getcwd(), os.pardir))
sys.path.append(parent_dir)

In [3]:
import torch
import torchvision
import torchvision.transforms as transforms
from utils import trainer as t
from utils import agop_conv as agc
from torch.utils.data import Dataset
import random
import torch.backends.cudnn as cudnn
import rfm
import numpy as np
import visdom
import model2
import numpy as np
from sklearn.model_selection import train_test_split
from torch.linalg import norm
from torchvision import models
import torch.optim as optim
import torch.nn.functional as F
import torch.nn as nn
from copy import deepcopy

In [5]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
#device='cpu'
print(f"Using device: {device}")

Using device: cuda:0


In [7]:
torch.cuda.empty_cache()

In [9]:
transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.1307,), (0.3081,))  # Mean and standard deviation for MNIST
    ])

trainset = torchvision.datasets.MNIST(root='./data', train=True, download=True, transform=transform)
trainset, valset = train_test_split(trainset, train_size=0.8)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=64, shuffle=False, num_workers=2)
valloader = torch.utils.data.DataLoader(valset, batch_size=100,
                                            shuffle=False, num_workers=1)

testset = torchvision.datasets.MNIST(root='./data', train=False, download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=64, shuffle=False, num_workers=2)

In [11]:
net= model2.ConvNet()
#Should Deep copy and try it
init_net=deepcopy(net)

In [27]:
import os
if os.path.exists('nn_models/mnist_conv_trained_nn.pth'):
    checkpoint = torch.load('nn_models/mnist_conv_trained_nn.pth', map_location=torch.device(device))
    net.load_state_dict(checkpoint['state_dict'])  # Access the 'state_dict' within the loaded dictionary
    print("Model weights loaded successfully.")

t.train_network(trainloader, valloader, testloader,
                    num_classes=10, optimizer= optim.Adam(net.parameters(), lr=0.001), 
                    lfn= nn.CrossEntropyLoss(),
                    name='mnist_conv', net=net)

Model weights loaded successfully.
NUMBER OF PARAMS:  430196
Time:  16.231115579605103
Epoch:  0 Train Loss:  0.023309473297083362 Test Loss:  0.041507467118278144 Train Acc:  99.34791666666666 Test Acc:  98.74 Best Val Acc:  98.975 Best Val Loss:  0.03006661233569806 Best Test Acc:  98.74 Best Test Loss:  0.041507467118278144
Time:  19.735631227493286
Epoch:  1 Train Loss:  0.012227988007575429 Test Loss:  0.03507194934821164 Train Acc:  99.69583333333334 Test Acc:  99.04 Best Val Acc:  99.31666666666666 Best Val Loss:  0.023046843872240666 Best Test Acc:  99.04 Best Test Loss:  0.03507194934821164
Time:  19.41743779182434
Epoch:  2 Train Loss:  0.00922354183485125 Test Loss:  0.03935641756817349 Train Acc:  99.44583333333334 Test Acc:  98.82 Best Val Acc:  99.31666666666666 Best Val Loss:  0.023046843872240666 Best Test Acc:  99.04 Best Test Loss:  0.03507194934821164
Time:  18.556450843811035
Epoch:  3 Train Loss:  0.009063903665075486 Test Loss:  0.041211003612149764 Train Acc:  99

# Verify NFA for conv layers:

In [34]:
# Here layer_idx refers to layer_idx+1 th conv layer. 
agc.verify_NFA(net.to(device), init_net.to(device), trainloader, layer_idx=1)

Correlation between Initial and Trained CNFM:  tensor(0.2035, device='cuda:0', grad_fn=<DivBackward0>)
Computing GOP for sample 0 out of 10
Computing GOP for sample 1 out of 10
Computing GOP for sample 2 out of 10
Computing GOP for sample 3 out of 10
Computing GOP for sample 4 out of 10
Computing GOP for sample 5 out of 10
Computing GOP for sample 6 out of 10
Computing GOP for sample 7 out of 10
Computing GOP for sample 8 out of 10
Computing GOP for sample 9 out of 10
Computing GOP for sample 10 out of 10
Correlation between Trained CNFM and AGOP:  tensor(0.9072, device='cuda:0')
Final:  tensor(0.2035, device='cuda:0', grad_fn=<DivBackward0>) tensor(0.9072, device='cuda:0')
